In [10]:
import oci
import os
from oci.object_storage import UploadManager
from oci.object_storage.models import CreateBucketDetails
from oci.object_storage.transfer.constants import MEBIBYTE
import filecmp

In [2]:
#source of connection to OCI is local config file, example in repo
config = oci.config.from_file('/Users/enjli/Angie/config')
identity = oci.identity.IdentityClient(config)
oci.config.validate_config(config)
user = identity.get_user(config["user"]).data
compartment_id = config["tenancy"]

In [3]:
#check
print(user)

{
  "capabilities": {
    "can_use_api_keys": true,
    "can_use_auth_tokens": true,
    "can_use_console_password": true,
    "can_use_customer_secret_keys": true,
    "can_use_smtp_credentials": true
  },
  "compartment_id": "ocid1.tenancy.oc1..aaaaaaaa62o4ctw2vulxg73pcv3yxjkbxvhjd3iq4xhi7ihunjr5tuqyf67a",
  "defined_tags": {},
  "description": "Enjing Li",
  "email": "enjing9541@hotmail.com",
  "external_identifier": null,
  "freeform_tags": {},
  "id": "ocid1.user.oc1..aaaaaaaapdspoz7t7irwlii3oxiyuvxfsgcpqy4xuaq4otukceaehz6aczca",
  "identity_provider_id": null,
  "inactive_status": null,
  "is_mfa_activated": false,
  "lifecycle_state": "ACTIVE",
  "name": "enjing9541@hotmail.com",
  "time_created": "2020-02-13T00:28:10.550000+00:00"
}


In [18]:
#selecting/creating object storage bucket and naming the bucket and object
object_storage = oci.object_storage.ObjectStorageClient(config)
namespace = object_storage.get_namespace().data
bucket_name = "bucket-20200227-1414"
object_name = "Model.pkl"

In [ ]:
#creates the bucket
#don't need to run if a bucket already exists
print("Creating a new bucket {!r} in compartment {!r}".format(bucket_name, compartment_id))
request = CreateBucketDetails()
request.compartment_id = compartment_id
request.name = bucket_name
bucket = object_storage.create_bucket(namespace, request)

In [19]:
with open('/Users/enjli/github/investbot/saved_model.pkl', 'rb') as f:
    file = f.read()
filename = "saved_model.pkl"
#needed for upload
filepath = '/Users/enjli/github/investbot/saved_model.pkl'

In [20]:
#various parameters to do multi-part uploading to object storage
#this is mostly automated in the oci console, meaning that is "generally" better to simply use the console to upload objects
# To force single part uploads, set "allow_multipart_uploads=False" when creating the UploadManager.
part_size = 2 * MEBIBYTE  # part size (in bytes)
upload_manager = UploadManager(object_storage, allow_parallel_uploads=True, parallel_process_count=3)
response = upload_manager.upload_file(
    namespace, bucket_name, object_name, filepath, part_size=part_size)

In [21]:
# Retrieve the file, streaming it into another file in 1 MiB chunks
print('Retrieving file from object storage')
get_obj = object_storage.get_object(namespace, bucket_name, object_name)


Retrieving file from object storage


In [22]:
with open('file_retrieved', 'wb') as f:
    for chunk in get_obj.data.raw.stream(1024 * 1024, decode_content=False):
        f.write(chunk)

print('Uploaded and downloaded files are the same: {}'.format(filecmp.cmp('saved_model.pkl', 'file_retrieved')))

Uploaded and downloaded files are the same: True
